# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 20 2022 12:44PM,239966,10,MSP211433,"Methapharm, Inc.",Released
1,Jun 20 2022 12:44PM,239965,19,8170014,"Zymergen, Inc.",Released
2,Jun 20 2022 12:19PM,239959,10,PHOX-705080,"Herbivore Botanicals, LLC.",Released
3,Jun 20 2022 12:12PM,239956,20,8168096,"Exact-Rx, Inc.",Released
4,Jun 20 2022 12:12PM,239956,20,8169329,"Exact-Rx, Inc.",Released
5,Jun 20 2022 12:12PM,239956,20,8169330,"Exact-Rx, Inc.",Released
6,Jun 20 2022 12:12PM,239956,20,8169331,"Exact-Rx, Inc.",Released
7,Jun 20 2022 12:12PM,239956,20,8169332,"Exact-Rx, Inc.",Released
8,Jun 20 2022 12:12PM,239956,20,8169333,"Exact-Rx, Inc.",Released
9,Jun 20 2022 12:12PM,239956,20,8169334,"Exact-Rx, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
44,239956,Released,22
45,239957,Released,1
46,239959,Released,1
47,239965,Released,1
48,239966,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239956,NaN,NaN,22.0
239957,NaN,NaN,1.0
239959,NaN,NaN,1.0
239965,NaN,NaN,1.0
239966,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239878,0.0,0.0,1.0
239892,0.0,0.0,1.0
239896,0.0,0.0,1.0
239897,0.0,0.0,1.0
239898,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239878,0,0,1
239892,0,0,1
239896,0,0,1
239897,0,0,1
239898,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239878,0,0,1
1,239892,0,0,1
2,239896,0,0,1
3,239897,0,0,1
4,239898,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239878,,,1
1,239892,,,1
2,239896,,,1
3,239897,,,1
4,239898,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 20 2022 12:44PM,239966,10,"Methapharm, Inc."
1,Jun 20 2022 12:44PM,239965,19,"Zymergen, Inc."
2,Jun 20 2022 12:19PM,239959,10,"Herbivore Botanicals, LLC."
3,Jun 20 2022 12:12PM,239956,20,"Exact-Rx, Inc."
25,Jun 20 2022 12:12PM,239955,10,ISDIN Corporation
77,Jun 20 2022 12:10PM,239957,12,"SD Head USA, LLC"
78,Jun 20 2022 12:01PM,239953,20,"ACI Healthcare USA, Inc."
107,Jun 20 2022 11:58AM,239952,19,"Innogenix, LLC"
130,Jun 20 2022 11:20AM,239949,10,Bio-PRF
158,Jun 20 2022 11:18AM,239950,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 20 2022 12:44PM,239966,10,"Methapharm, Inc.",,,1
1,Jun 20 2022 12:44PM,239965,19,"Zymergen, Inc.",,,1
2,Jun 20 2022 12:19PM,239959,10,"Herbivore Botanicals, LLC.",,,1
3,Jun 20 2022 12:12PM,239956,20,"Exact-Rx, Inc.",,,22
4,Jun 20 2022 12:12PM,239955,10,ISDIN Corporation,,,52
5,Jun 20 2022 12:10PM,239957,12,"SD Head USA, LLC",,,1
6,Jun 20 2022 12:01PM,239953,20,"ACI Healthcare USA, Inc.",,,29
7,Jun 20 2022 11:58AM,239952,19,"Innogenix, LLC",,17,6
8,Jun 20 2022 11:20AM,239949,10,Bio-PRF,,,28
9,Jun 20 2022 11:18AM,239950,10,Emerginnova,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 20 2022 12:44PM,239966,10,"Methapharm, Inc.",1,,
1,Jun 20 2022 12:44PM,239965,19,"Zymergen, Inc.",1,,
2,Jun 20 2022 12:19PM,239959,10,"Herbivore Botanicals, LLC.",1,,
3,Jun 20 2022 12:12PM,239956,20,"Exact-Rx, Inc.",22,,
4,Jun 20 2022 12:12PM,239955,10,ISDIN Corporation,52,,
5,Jun 20 2022 12:10PM,239957,12,"SD Head USA, LLC",1,,
6,Jun 20 2022 12:01PM,239953,20,"ACI Healthcare USA, Inc.",29,,
7,Jun 20 2022 11:58AM,239952,19,"Innogenix, LLC",6,17,
8,Jun 20 2022 11:20AM,239949,10,Bio-PRF,28,,
9,Jun 20 2022 11:18AM,239950,10,Emerginnova,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 20 2022 12:44PM,239966,10,"Methapharm, Inc.",1,,
1,Jun 20 2022 12:44PM,239965,19,"Zymergen, Inc.",1,,
2,Jun 20 2022 12:19PM,239959,10,"Herbivore Botanicals, LLC.",1,,
3,Jun 20 2022 12:12PM,239956,20,"Exact-Rx, Inc.",22,,
4,Jun 20 2022 12:12PM,239955,10,ISDIN Corporation,52,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 20 2022 12:44PM,239966,10,"Methapharm, Inc.",1.0,NaN,NaN
1,Jun 20 2022 12:44PM,239965,19,"Zymergen, Inc.",1.0,NaN,NaN
2,Jun 20 2022 12:19PM,239959,10,"Herbivore Botanicals, LLC.",1.0,NaN,NaN
3,Jun 20 2022 12:12PM,239956,20,"Exact-Rx, Inc.",22.0,NaN,NaN
4,Jun 20 2022 12:12PM,239955,10,ISDIN Corporation,52.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 20 2022 12:44PM,239966,10,"Methapharm, Inc.",1.0,0.0,0.0
1,Jun 20 2022 12:44PM,239965,19,"Zymergen, Inc.",1.0,0.0,0.0
2,Jun 20 2022 12:19PM,239959,10,"Herbivore Botanicals, LLC.",1.0,0.0,0.0
3,Jun 20 2022 12:12PM,239956,20,"Exact-Rx, Inc.",22.0,0.0,0.0
4,Jun 20 2022 12:12PM,239955,10,ISDIN Corporation,52.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4318799,253.0,29.0,0.0
12,239957,1.0,0.0,0.0
15,719771,19.0,48.0,11.0
16,1199512,5.0,0.0,0.0
18,479844,2.0,0.0,0.0
19,959759,8.0,18.0,0.0
20,719808,66.0,0.0,0.0
21,479856,2.0,0.0,0.0
22,959739,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4318799,253.0,29.0,0.0
1,12,239957,1.0,0.0,0.0
2,15,719771,19.0,48.0,11.0
3,16,1199512,5.0,0.0,0.0
4,18,479844,2.0,0.0,0.0
5,19,959759,8.0,18.0,0.0
6,20,719808,66.0,0.0,0.0
7,21,479856,2.0,0.0,0.0
8,22,959739,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,253.0,29.0,0.0
1,12,1.0,0.0,0.0
2,15,19.0,48.0,11.0
3,16,5.0,0.0,0.0
4,18,2.0,0.0,0.0
5,19,8.0,18.0,0.0
6,20,66.0,0.0,0.0
7,21,2.0,0.0,0.0
8,22,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,253.0
1,12,Released,1.0
2,15,Released,19.0
3,16,Released,5.0
4,18,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Completed,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,29.0,0.0,48.0,0.0,0.0,18.0,0.0,0.0,0.0
Released,253.0,1.0,19.0,5.0,2.0,8.0,66.0,2.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Completed,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,29.0,0.0,48.0,0.0,0.0,18.0,0.0,0.0,0.0
2,Released,253.0,1.0,19.0,5.0,2.0,8.0,66.0,2.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Completed,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,29.0,0.0,48.0,0.0,0.0,18.0,0.0,0.0,0.0
2,Released,253.0,1.0,19.0,5.0,2.0,8.0,66.0,2.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()